This notebook provides a convenient wrapper for bag_reader.  
It looks for all bagfiles in a specified directory and converts to pickle/matfile for further processing.

To handle various parsing errors, we use a try/except block and only save pkl files corresponding to valid rosbag data.

In [1]:
import glob
import pickle
import scipy.io as sio
from bag_reader import *
from tqdm import tqdm
import multiprocessing

Failed to load Python extension for LZ4 support. LZ4 compression will not be available.
/home/mpc/carla/PythonAPI/analysis/transformations.py:1926: UserWarning: No module named '_transformations'
  warnings.warn(str(err))


In [2]:
# bagfiles_to_process = glob.glob('../examples/bags/*.bag')
bagfiles_to_process = glob.glob('/media/data/carla_parking_data_bkcp/bags/*.bag')

# bagfiles_to_process = glob.glob('/media/data/carla_parking_data_bkcp/bags_original/*.bag')

print('Found %d bagfiles: %s' % (len(bagfiles_to_process), bagfiles_to_process))


Found 595 bagfiles: ['/media/data/carla_parking_data_bkcp/bags/parking_p8_t6_e7_2020-01-12-14-10-45.bag', '/media/data/carla_parking_data_bkcp/bags/parking_p5_t6_e2_2020-01-08-14-58-59.bag', '/media/data/carla_parking_data_bkcp/bags/parking_p6_t4_e6_2020-01-09-16-14-44.bag', '/media/data/carla_parking_data_bkcp/bags/parking_p12_t4_e0_2020-01-14-15-59-08.bag', '/media/data/carla_parking_data_bkcp/bags/parking_p13_t3_e8_2020-01-15-10-14-11.bag', '/media/data/carla_parking_data_bkcp/bags/parking_p6_t3_e2_2020-01-09-16-08-47.bag', '/media/data/carla_parking_data_bkcp/bags/parking_p5_t1_e3_2020-01-08-14-26-24.bag', '/media/data/carla_parking_data_bkcp/bags/parking_p11_t4_e6_2020-01-14-14-05-04.bag', '/media/data/carla_parking_data_bkcp/bags/parking_p13_t6_e1_2020-01-15-10-23-06.bag', '/media/data/carla_parking_data_bkcp/bags/parking_p8_t6_e5_2020-01-12-14-09-42.bag', '/media/data/carla_parking_data_bkcp/bags/parking_p6_t5_e7_2020-01-09-18-35-26.bag', '/media/data/carla_parking_data_bkcp/bag

### Topics processed by bag_reader.py

* [x] /carla/actor_list
* [x] /carla/hero/collision
* [x] /carla/hero/gnss/front/fix
* [x] /carla/hero/objects
* [x] /carla/hero/odometry
* [x] /carla/hero/vehicle_info
* [x] /carla/hero/vehicle_status
* [x] /carla/odometry
* [x] /carla/status
* [x] /carla/world_info
* [x] /intention
* [ ] <span style="color:blue"> /carla/objects </span>
* [ ] <span style="color:blue"> /rosout </span>
* [ ] <span style="color:blue"> /rosout_agg </span>
* [ ] <span style="color:blue"> /tf </span>
* [ ] <span style="color:blue"> /carla/camera/rgb/front/camera_info </span>
* [ ] <span style="color:blue"> /carla/camera/rgb/front/image_color </span>
* [ ] <span style="color:blue"> /carla/marker </span>
* [ ] <span style="color:blue"> /clock </span>

### Additional Notes
<span style="color:blue"> We do not plan to process blue topics (or will do it separately for images). /carla/objects contains the ego vehicle as well, so excluded. </span>

<span style="color:red;font-weight:bold"> Outlier detection is currently only applied for the ego odometry.</span>


In [3]:
def process_bag_helper(bagname):
    print('Processing ', bagname)
    try:
        result_dict = process_bag(bagname)
        fname = bagname.split('.bag')[0]
        assert('parking' in fname)

        '''
        # Save to file (matfile, needs work)
        fname += '.mat'
        print('Saving ', fname)
        sio.savemat(fname, result_dict)
        '''

        fname += '.pkl'
        pickle.dump(result_dict, open(fname, 'wb'))
        print('Saved :', fname)
        
    except Exception as e:
        print('Error: ', e)

with multiprocessing.Pool(50) as p:
    p.map(process_bag_helper, bagfiles_to_process)

Processing  /media/data/carla_parking_data_bkcp/bags/parking_p8_t6_e7_2020-01-12-14-10-45.bag
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p9_t5_e5_2020-01-13-12-43-12.bag
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p8_t6_e5_2020-01-12-14-09-42.bag
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p12_t4_e0_2020-01-14-15-59-08.bag
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p12_t1_e4_2020-01-14-15-42-28.bag
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p12_t1_e6_2020-01-14-15-43-44.bag
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p5_t1_e3_2020-01-08-14-26-24.bag
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p9_t4_e1_2020-01-13-12-36-11.bag
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p7_t2_e5_2020-01-10-14-57-49.bag
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p11_t2_e2_2020-01-14-13-40-14.bag
Processing  /media/data/carla_parking_data_bkcp/bags/par

	 File: parking_p10_t3_e3_2020-01-13-14-55-30, Start point processed. Setting start time 5437.503 vs. first entry at time 5437.503 for ego odom.
Saved : /media/data/carla_parking_data_bkcp/bags/parking_p10_t3_e3_2020-01-13-14-55-30.pkl
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p13_t4_e2_2020-01-15-10-17-05.bag
	 File: parking_p13_t2_e9_2020-01-15-10-11-46, Start point processed. Setting start time 3392.506 vs. first entry at time 3392.506 for ego odom.
Saved : /media/data/carla_parking_data_bkcp/bags/parking_p13_t2_e9_2020-01-15-10-11-46.pkl
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p5_t3_e2_2020-01-08-14-41-54.bag
	 File: parking_p9_t3_e7_2020-01-13-12-33-39, Start point processed. Setting start time 5806.087 vs. first entry at time 5806.087 for ego odom.
Saved : /media/data/carla_parking_data_bkcp/bags/parking_p9_t3_e7_2020-01-13-12-33-39.pkl
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p11_t5_e0_2020-01-14-14-08-37.bag
	 File: pa

Processing  /media/data/carla_parking_data_bkcp/bags/parking_p13_t5_e5_2020-01-15-10-21-07.bag
Error:  	No intention time, so skipping this instance.
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p12_t1_e1_2020-01-14-15-40-30.bag
	 File: parking_p13_t4_e2_2020-01-15-10-17-05, Start point processed. Setting start time 274.759 vs. first entry at time 274.759 for ego odom.
Saved : /media/data/carla_parking_data_bkcp/bags/parking_p13_t4_e2_2020-01-15-10-17-05.pkl
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p11_t4_e3_2020-01-14-14-02-51.bag
	 File: parking_p13_t1_e1_2020-01-15-10-04-17, Start point processed. Setting start time 2562.272 vs. first entry at time 2562.272 for ego odom.
Saved : /media/data/carla_parking_data_bkcp/bags/parking_p13_t1_e1_2020-01-15-10-04-17.pkl
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p9_t4_e6_2020-01-13-12-38-39.bag
	 File: parking_p9_t4_e4_2020-01-13-12-37-21, Start point processed. Setting start time 7107.154

Saved : /media/data/carla_parking_data_bkcp/bags/parking_p6_t3_e9_2020-01-09-16-11-14.pkl
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p9_t3_e1_2020-01-13-12-31-23.bag
	 File: parking_p6_t1_e1_2020-01-09-15-53-42, Start point processed. Setting start time 9538.401 vs. first entry at time 9538.401 for ego odom.
Saved : /media/data/carla_parking_data_bkcp/bags/parking_p6_t1_e1_2020-01-09-15-53-42.pkl
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p5_t2_e4_2020-01-08-14-34-40.bag
Error:  Ego vehicle not found in object list!
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p13_t4_e6_2020-01-15-10-18-20.bag
	 File: parking_p10_t2_e5_2020-01-13-14-48-11, Start point processed. Setting start time 3933.903 vs. first entry at time 3933.903 for ego odom.
Saved : /media/data/carla_parking_data_bkcp/bags/parking_p10_t2_e5_2020-01-13-14-48-11.pkl
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p6_t5_e3_2020-01-09-18-33-53.bag
	 File: parking_p

	 File: parking_p5_t6_e9_2020-01-08-15-03-05, Start point processed. Setting start time 25436.926 vs. first entry at time 25436.926 for ego odom.
Saved : /media/data/carla_parking_data_bkcp/bags/parking_p5_t6_e9_2020-01-08-15-03-05.pkl
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p9_t2_e9_2020-01-13-12-30-12.bag
	 File: parking_p5_t6_e2_2020-01-08-14-58-59, Start point processed. Setting start time 25215.426 vs. first entry at time 25215.426 for ego odom.
Saved : /media/data/carla_parking_data_bkcp/bags/parking_p5_t6_e2_2020-01-08-14-58-59.pkl
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p6_t4_e6_2020-01-09-16-14-44.bag
	 File: parking_p6_t5_e3_2020-01-09-18-33-53, Start point processed. Setting start time 284482.849 vs. first entry at time 284482.849 for ego odom.
Saved : /media/data/carla_parking_data_bkcp/bags/parking_p6_t5_e3_2020-01-09-18-33-53.pkl
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p7_t1_e0_2020-01-10-14-49-53.bag
	 File: 

...

Processing  /media/data/carla_parking_data_bkcp/bags/parking_p13_t4_e3_2020-01-15-10-17-24.bag
	 File: parking_p6_t1_e8_2020-01-09-15-58-09, Start point processed. Setting start time 9775.468 vs. first entry at time 9775.468 for ego odom.
Saved : /media/data/carla_parking_data_bkcp/bags/parking_p6_t1_e8_2020-01-09-15-58-09.pkl
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p8_t3_e8_2020-01-12-13-54-56.bag
	 File: parking_p5_t5_e1_2020-01-08-14-53-37, Start point processed. Setting start time 24727.392 vs. first entry at time 24727.392 for ego odom.
Saved : /media/data/carla_parking_data_bkcp/bags/parking_p5_t5_e1_2020-01-08-14-53-37.pkl
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p14_t6_e3_2020-01-15-11-40-16.bag
	 File: parking_p5_t4_e5_2020-01-08-14-48-49, Start point processed. Setting start time 23760.126 vs. first entry at time 23760.126 for ego odom.
Saved : /media/data/carla_parking_data_bkcp/bags/parking_p5_t4_e5_2020-01-08-14-48-49.pkl
Processing

Processing  /media/data/carla_parking_data_bkcp/bags/parking_p5_t4_e4_2020-01-08-14-48-14.bag
	 File: parking_p8_t3_e8_2020-01-12-13-54-56, Start point processed. Setting start time 9110.880 vs. first entry at time 9110.880 for ego odom.
Saved : /media/data/carla_parking_data_bkcp/bags/parking_p8_t3_e8_2020-01-12-13-54-56.pkl
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p6_t4_e0_2020-01-09-16-11-58.bag
	 File: parking_p8_t3_e5_2020-01-12-13-53-10, Start point processed. Setting start time 9010.747 vs. first entry at time 9010.747 for ego odom.
Saved : /media/data/carla_parking_data_bkcp/bags/parking_p8_t3_e5_2020-01-12-13-53-10.pkl
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p5_t6_e7_2020-01-08-15-01-52.bag
	 File: parking_p7_t3_e7_2020-01-10-15-03-12, Start point processed. Setting start time 15350.932 vs. first entry at time 15350.932 for ego odom.
Saved : /media/data/carla_parking_data_bkcp/bags/parking_p7_t3_e7_2020-01-10-15-03-12.pkl
Processing  /m

Saved : /media/data/carla_parking_data_bkcp/bags/parking_p6_t2_e7_2020-01-09-16-03-30.pkl
	 File: parking_p11_t5_e4_2020-01-14-14-11-11, Start point processed. Setting start time 11714.408 vs. first entry at time 11714.408 for ego odom.
Saved : /media/data/carla_parking_data_bkcp/bags/parking_p11_t5_e4_2020-01-14-14-11-11.pkl
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p9_t3_e4_2020-01-13-12-32-31.bag
Error:  Parked vehicle has non-zero heading!
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p7_t5_e0_2020-01-15-11-06-06.bag
	 File: parking_p6_t2_e1_2020-01-09-16-00-17, Start point processed. Setting start time 10943.768 vs. first entry at time 10943.768 for ego odom.
Saved : /media/data/carla_parking_data_bkcp/bags/parking_p6_t2_e1_2020-01-09-16-00-17.pkl
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p8_t1_e8_2020-01-12-13-38-04.bag
Error:  Parked vehicle has non-zero heading!
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p10

	 File: parking_p6_t1_e2_2020-01-09-15-54-23, Start point processed. Setting start time 9575.168 vs. first entry at time 9575.168 for ego odom.
Saved : /media/data/carla_parking_data_bkcp/bags/parking_p6_t1_e2_2020-01-09-15-54-23.pkl
Error:  Parked vehicle has non-zero heading!
	 File: parking_p13_t6_e9_2020-01-15-10-25-51, Start point processed. Setting start time 1144.759 vs. first entry at time 1144.759 for ego odom.
Saved : /media/data/carla_parking_data_bkcp/bags/parking_p13_t6_e9_2020-01-15-10-25-51.pkl
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p7_t3_e8_2020-01-10-15-03-35.bag
	 File: parking_p13_t5_e0_2020-01-15-10-19-52, Start point processed. Setting start time 637.759 vs. first entry at time 637.759 for ego odom.
Saved : /media/data/carla_parking_data_bkcp/bags/parking_p13_t5_e0_2020-01-15-10-19-52.pkl
	 File: parking_p11_t1_e0_2020-01-14-13-28-48, Start point processed. Setting start time 5657.108 vs. first entry at time 5657.108 for ego odom.
Saved : /med

Saved : /media/data/carla_parking_data_bkcp/bags/parking_p7_t2_e2_2020-01-10-14-56-18.pkl
Error:  Ego vehicle not found in object list!
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p11_t2_e9_2020-01-14-13-48-57.bag
	 File: parking_p10_t1_e0_2020-01-13-14-39-05, Start point processed. Setting start time 3363.836 vs. first entry at time 3363.836 for ego odom.
Saved : /media/data/carla_parking_data_bkcp/bags/parking_p10_t1_e0_2020-01-13-14-39-05.pkl
	 File: parking_p7_t2_e4_2020-01-10-14-57-21, Start point processed. Setting start time 14152.866 vs. first entry at time 14152.866 for ego odom.
Saved : /media/data/carla_parking_data_bkcp/bags/parking_p7_t2_e4_2020-01-10-14-57-21.pkl
Processing  /media/data/carla_parking_data_bkcp/bags/parking_p13_t6_e0_2020-01-15-10-22-40.bag
	 File: parking_p11_t6_e8_2020-01-14-14-22-26, Start point processed. Setting start time 12684.175 vs. first entry at time 12684.175 for ego odom.
Saved : /media/data/carla_parking_data_bkcp/bags/parkin